In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
  

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()
train_data.keys()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [3]:
print("start")
#print how many nans in each columns
train_data['Embarked'].dropna(inplace=True)

for key in train_data.keys():
    print(key + str(train_data[key].isna().sum()))
    
print("done")

start
PassengerId0
Survived0
Pclass0
Name0
Sex0
Age177
SibSp0
Parch0
Ticket0
Fare0
Cabin687
Embarked0
done


Split into training and testing data set

In [4]:
x= train_data[['Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']]
y=train_data["Survived"]
x['Youth'] = 0
x.loc[x['Age'] <=16, 'Youth'] = 1

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
# Columns to be one-hot encoded
columns_to_encode = [ 'Embarked', 'Sex']

# Perform onefit-hot encoding
one_hot_encoded = pd.get_dummies(x[columns_to_encode])

# Remove the original columns
df_encoded = x.drop(columns_to_encode, axis=1)


# Concatenate the one-hot encoded columns to the modified DataFrame
df_encoded = pd.concat([df_encoded, one_hot_encoded], axis=1)
column = list(df_encoded.keys())
df_encoded = my_imputer.fit_transform(df_encoded)
df_encoded = pd.DataFrame(df_encoded, columns = column)

print(df_encoded)
# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(df_encoded,y ,
                                   random_state=1, 
                                   test_size=0.001, 
                                   shuffle=True)


print("done splitting into test an train set")

     Pclass        Age  SibSp  Parch     Fare  Youth  Embarked_C  Embarked_Q  \
0       3.0  22.000000    1.0    0.0   7.2500    0.0         0.0         0.0   
1       1.0  38.000000    1.0    0.0  71.2833    0.0         1.0         0.0   
2       3.0  26.000000    0.0    0.0   7.9250    0.0         0.0         0.0   
3       1.0  35.000000    1.0    0.0  53.1000    0.0         0.0         0.0   
4       3.0  35.000000    0.0    0.0   8.0500    0.0         0.0         0.0   
..      ...        ...    ...    ...      ...    ...         ...         ...   
886     2.0  27.000000    0.0    0.0  13.0000    0.0         0.0         0.0   
887     1.0  19.000000    0.0    0.0  30.0000    0.0         0.0         0.0   
888     3.0  29.699118    1.0    2.0  23.4500    0.0         0.0         0.0   
889     1.0  26.000000    0.0    0.0  30.0000    0.0         1.0         0.0   
890     3.0  32.000000    0.0    0.0   7.7500    0.0         0.0         1.0   

     Embarked_S  Sex_female  Sex_male  

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
smote = SMOTE()
print("1")
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
scaler = StandardScaler()
standardized_data = scaler.fit_transform(X_resampled)
lda = LDA()
print("1")

lda.fit(standardized_data, y_resampled)
# Make predictions on the testing data
x_test_standardized = scaler.transform(X_test)
print("1")

y_pred = lda.predict(x_test_standardized)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


output = pd.DataFrame({ 'Survived': y_pred})
output.to_csv('test_results.csv', index=False)
print("Your test was successfully saved!")


Using TensorFlow backend.


1
1
1
Accuracy: 1.0


/opt/conda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Your test was successfully saved!


In [6]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Before moving on, make sure that your code returns the output above.  The code above calculates the percentage of female passengers (in **train.csv**) who survived.

Then, run the code below in another code cell:

In [7]:
x_test= test_data[['Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']]
y_test=train_data["Survived"]
x_test.loc[x_test['Age'] <=16, 'Youth'] = 1

# Columns to be one-hot encoded
columns_to_encode = [ 'Embarked', 'Sex']

# Perform onefit-hot encoding
one_hot_encoded_test = pd.get_dummies(x_test[columns_to_encode])

# Remove the original columns
df_encoded_test = x_test.drop(columns_to_encode, axis=1)


# Concatenate the one-hot encoded columns to the modified DataFrame
df_encoded_test= pd.concat([df_encoded_test, one_hot_encoded_test], axis=1)
df_encoded_test = df_encoded_test.reindex(columns = df_encoded.columns, fill_value=0)

column = list(df_encoded_test.keys())
df_encoded_test = my_imputer.transform(df_encoded_test)
df_encoded_test = pd.DataFrame(df_encoded_test, columns = column)

print(df_encoded_test)
df_encoded_test  = scaler.transform(df_encoded_test)
print("done splitting into test an train set")
predictions = lda.predict(df_encoded_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print(output)
print("Your submission was successfully saved!")

     Pclass        Age  SibSp  Parch      Fare     Youth  Embarked_C  \
0       3.0  34.500000    0.0    0.0    7.8292  0.112233         0.0   
1       3.0  47.000000    1.0    0.0    7.0000  0.112233         0.0   
2       2.0  62.000000    0.0    0.0    9.6875  0.112233         0.0   
3       3.0  27.000000    0.0    0.0    8.6625  0.112233         0.0   
4       3.0  22.000000    1.0    1.0   12.2875  0.112233         0.0   
..      ...        ...    ...    ...       ...       ...         ...   
413     3.0  29.699118    0.0    0.0    8.0500  0.112233         0.0   
414     1.0  39.000000    0.0    0.0  108.9000  0.112233         1.0   
415     3.0  38.500000    0.0    0.0    7.2500  0.112233         0.0   
416     3.0  29.699118    0.0    0.0    8.0500  0.112233         0.0   
417     3.0  29.699118    1.0    1.0   22.3583  0.112233         1.0   

     Embarked_Q  Embarked_S  Sex_female  Sex_male  
0           1.0         0.0         0.0       1.0  
1           0.0         1.0    

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier()
clf.fit(standardized_data, y_resampled)

predictions = clf.predict(df_encoded_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission_random_forest.csv', index=False)
print(output)
print("Your submission was successfully saved!")

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
Your submission was successfully saved!


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
